In [2]:
!pip install pandas
!pip install matplotlib

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 2.6 MB/s eta 0:00:00 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 8.9 MB/s eta 0:00:00ta 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 14.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 14.0 MB/s eta 0:00:00a 0:00:01
Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 [matplotlib]7 [matplotlib]


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [77]:
# csv_files = [
#     'qmlp-mnist-run_run1-layer2_all_attacks.csv',
#     'qmlp-mnist-run_run2-layer2_all_attacks.csv',
#     'qmlp-mnist-run_run3-layer2_all_attacks.csv'
# ]

# csv_files = [
#     'qmlp-mnist-run_run1-layer5_all_attacks.csv',
#     'qmlp-mnist-run_run2-layer5_all_attacks.csv',
#     'qmlp-mnist-run_run3-layer5_all_attacks.csv'
# ]

csv_files = [
    'qmlp-mnist-run_run1-layer10_all_attacks.csv',
    'qmlp-mnist-run_run2-layer10_all_attacks.csv',
    'qmlp-mnist-run_run3-layer10_all_attacks.csv'
]

columns_to_analyze = [
'accuracy', 'loss', 'precision', 'recall', 'f1', 'fpr', 'fnr', 'roc_auc', 'pr_auc'
]
ndfs = []
for file in csv_files:
    df = pd.read_csv(file)
    ndfs.append(df)
combined_df = pd.concat(ndfs, ignore_index=True)  

In [78]:
print(combined_df.head())
print(len(combined_df))

   Run ID  Epsilon Attack Type  accuracy      loss  precision    recall  \
0       1     0.00        none    0.8206  0.583229   0.820392  0.818397   
1       1     0.00        fgsm    0.8206  0.583229   0.820392  0.818397   
2       1     0.01        fgsm    0.4369  1.880219   0.469440  0.444013   
3       1     0.10        fgsm    0.0273  5.241955   0.019570  0.026841   
4       1     0.15        fgsm    0.0091  7.037966   0.004951  0.008868   

         f1       fpr       fnr   roc_auc  pr_auc  
0  0.819393  0.019915  0.181603  0.978329     NaN  
1  0.819393  0.019915  0.181603  0.978329     NaN  
2  0.456373  0.062443  0.555987  0.856399     NaN  
3  0.022636  0.107816  0.973159  0.419013     NaN  
4  0.006354  0.109871  0.991132  0.312546     NaN  
27


In [79]:
def compute_mean_std(attack) -> None:
    attack_df = combined_df[combined_df['Attack Type'] == attack]
    
    # Store epsilons in a dictionary
    epsilons = {
        'eps1': attack_df[attack_df['Epsilon'] == 0.00],
        'eps2': attack_df[attack_df['Epsilon'] == 0.01],
        'eps3': attack_df[attack_df['Epsilon'] == 0.10],
        'eps4': attack_df[attack_df['Epsilon'] == 0.15],
    }
    
    for name, df in epsilons.items():
        # Drop non-metric columns
        metrics_df = df.drop(columns=['Run ID', 'Epsilon', 'Attack Type', 'pr_auc'], errors='ignore')
        
        # Calculate mean and std
        mean_series = metrics_df.mean()
        std_series = metrics_df.std()
        
        # Combine into a single DataFrame
        result_df = pd.DataFrame([mean_series, std_series], index=['mean', 'std'])
        
        print(f"\n{name} (Epsilon = {df['Epsilon'].iloc[0]}):")
        print(result_df)

print('FGSM')
compute_mean_std('fgsm')

FGSM

eps1 (Epsilon = 0.0):
      accuracy      loss  precision    recall        f1       fpr       fnr  \
mean  0.829833  0.558837   0.829817  0.827443  0.828628  0.018900  0.172557   
std   0.008670  0.024681   0.009018  0.008550  0.008782  0.000955  0.008550   

       roc_auc  
mean  0.980076  
std   0.001754  

eps2 (Epsilon = 0.01):
      accuracy      loss  precision    recall        f1       fpr       fnr  \
mean  0.414033  2.014737   0.482341  0.420059  0.448241  0.064987  0.579941   
std   0.031270  0.145117   0.013083  0.032611  0.014036  0.003500  0.032611   

       roc_auc  
mean  0.851513  
std   0.007821  

eps3 (Epsilon = 0.1):
      accuracy      loss  precision    recall        f1       fpr       fnr  \
mean  0.028167  5.642596   0.018743  0.027670  0.022186  0.107726  0.972330   
std   0.002136  0.349836   0.002612  0.002012  0.001315  0.000191  0.002012   

       roc_auc  
mean  0.417705  
std   0.002237  

eps4 (Epsilon = 0.15):
      accuracy      loss  precisio

In [80]:
print('PGD')
compute_mean_std('pgd')

PGD

eps1 (Epsilon = 0.0):
      accuracy      loss  precision    recall        f1       fpr       fnr  \
mean  0.829833  0.558837   0.829817  0.827443  0.828628  0.018900  0.172557   
std   0.008670  0.024681   0.009018  0.008550  0.008782  0.000955  0.008550   

       roc_auc  
mean  0.980076  
std   0.001754  

eps2 (Epsilon = 0.01):
      accuracy      loss  precision    recall        f1       fpr       fnr  \
mean  0.411200  2.026243   0.492711  0.417257  0.450989  0.065296  0.582743   
std   0.030335  0.149368   0.015713  0.031542  0.012687  0.003394  0.031542   

       roc_auc  
mean  0.854063  
std   0.007777  

eps3 (Epsilon = 0.1):
      accuracy      loss  precision    recall        f1       fpr       fnr  \
mean    0.0210  6.206117   0.018334  0.020625  0.019321  0.108553  0.979375   
std     0.0035  0.439014   0.004529  0.003393  0.003718  0.000283  0.003393   

       roc_auc  
mean  0.448725  
std   0.005459  

eps4 (Epsilon = 0.15):
      accuracy      loss  precision